# Core Update

This Notebook focuses on the process of modifying data.

The following topics are covered:
- Values
- Returning
- Update ... From


Further Reading:
- [UPDATE syntax](https://www.sqlite.org/lang_update.html) by SQLite
- ["Using UPDATE and DELETE Statements"](https://docs.sqlalchemy.org/en/20/tutorial/data_update.html) by SQLAlchemy

# Setup

In [ ]:
import sqlalchemy as sa
from utils import *

base = sa.MetaData()
Products = sa.Table(
    'products',
    base,
    sa.Column('id', sa.INTEGER, primary_key=True, autoincrement=True),
    sa.Column('name', sa.VARCHAR(255), nullable=False, index=True),
    sa.Column('price', sa.DOUBLE, nullable=True),
    sa.Column('extra', sa.DOUBLE, nullable=True)
)
Staging = sa.Table(
    'staging',
    base,
    sa.Column('id', sa.INTEGER, primary_key=True, autoincrement=False), # -- False
    sa.Column('name', sa.VARCHAR(255), nullable=False, index=True),
    sa.Column('price', sa.DOUBLE, nullable=True),
    sa.Column('extra', sa.DOUBLE, nullable=True)
)

engine = sa.create_engine('sqlite:///')
con = engine.connect()
base.create_all(engine)

# Basic Update

In [ ]:
with con.begin() as t:
    print('-- Inserting records --')
    con.execute(Products.insert(), [{'name': 'Apple'}, {'name': 'Android'}])
    for r in con.execute(Products.select()).mappings():
        print(r)

    print('\n-- Updating records (Android) --')
    result = con.execute(Products.update().values(price=400).where(Products.c.name=='Android'))
    print(f'{result.rowcount} row(s) affected.')
    for r in con.execute(Products.select()).mappings():
        print(r)
    print('\n-- Updating records (Extra) --')
    with logs():
        con.execute(Products.update().values(extra=Products.c.price).values(price=80))
    print('---')
    for r in con.execute(Products.select()).mappings():
        print(r)
    t.rollback()

# Values

Products.insert().values(column=value)

The `values()` method can be repeated, with each call adding columns and values to the query.<br>
When a column is already part of the update statement, the old value is overwritten.


## A bit more rigid


In [ ]:
with rollback(con), logs():
    _price = Products.c.price.key
    data = {_price:400}
    print('\n-- Updating records (Android) --')
    
    result = con.execute(Products.update().values(name='Apple').values(data).values(name='XYZ').where(Products.c.name=='Android'))

# Returning


# Update From
Most DBMS support updating with some sort of join in it.<br>
It is usually written as `UPDATE ... FROM`.<br>
Traditionally, this would us a subquery, but Update-From is becoming commonplace.

- [Update From at SQLAlchemy](https://docs.sqlalchemy.org/en/20/tutorial/data_update.html#update-from)

This uses a WHERE statement that effectively acts like a join.<br>
The following query performs the following action:
1. Create an apple.
2. Copy it from products to Staging.
3. Apply an uppercase transformation to all data on staging
4. Apply the update to Products, *from* staging.

In [ ]:
with rollback(con):
    con.execute(sa.insert(Products), {'name': 'Apple'})
    
    for entry in con.execute(sa.select(Products)):
        print(entry)

    # INSERT INTO staging SELECT [...] FROM Products
    q = (
        sa.insert(Staging)
        .from_select([Products.c.id, Products.c.name, Products.c.price, Products.c.extra], Products)
    )
    con.execute(q)

    # Put the names to uppercase (in staging)
    q = sa.update(Staging).values(name=sa.func.upper(Staging.c.name))
    con.execute(q)

    # Copy the data back to Products
    q = sa.update(Products).where(Staging.c.id == Products.c.id).values(name=Staging.c.name)
    with logs():
        con.execute(q)
    for entry in con.execute(sa.select(Products)):
        print(entry)